In [14]:
import graphviz
from datetime import datetime
import json
import numpy as np
import time

In [15]:
names = ['smallRandom','xsmallComplex','smallComplex','mediumRandom','MediumComplex','largeComplex','xlargeComplex','xxlargeComplex']

In [16]:
with open('./graphs/smallRandom.json', 'r') as f:
    data = json.load(f)

nodes = data['nodes']
print(nodes)

{'1': {'Data': '00:47:23.6552354', 'Dependencies': []}, '2': {'Data': '01:00:56.0712509', 'Dependencies': [1]}, '3': {'Data': '00:45:41.9822376', 'Dependencies': [1]}, '4': {'Data': '01:09:26.2243330', 'Dependencies': [1]}, '5': {'Data': '01:24:25.7910236', 'Dependencies': [2]}, '6': {'Data': '01:25:16.0078169', 'Dependencies': [4]}, '7': {'Data': '01:04:38.7031112', 'Dependencies': [2]}, '8': {'Data': '00:59:56.1902359', 'Dependencies': [5]}, '9': {'Data': '01:27:32.5795340', 'Dependencies': [7]}, '10': {'Data': '00:48:03.6393571', 'Dependencies': [8]}}


Since the formula of the rank is $rank(i) = p_i + max_{j \in succ(i)}(rank(j))$, it is more efficient to know both the predecessor and the successor of a node $i$.

In [17]:
def create_successors():
    for node in nodes:
        nodes[node]['Successors']=[]
    for node in nodes :
        depend = nodes[node]['Dependencies']
        for i in depend :
            nodes[str(i)]['Successors'].append(node)

In [18]:
def weight(node):
    pt = node['Data'].split(':')
    time_s = float(pt[0])*3600 + float(pt[1])*60 + float(pt[2])
    return time_s

In [19]:
def add_ranks():
    for node in nodes:
        nodes[node]['rank'] = weight(nodes[node])
        nodes[node]['rank_calculated'] = False

# for node in nodes.values() :
#     print(node['rank'])
#     print(node['rank_calculated'])

In [20]:
def calculate_rank(node):
    if not nodes[node]['rank_calculated']:
        succ = nodes[node]['Successors']     
        rank = weight(nodes[node])
        if len(succ) > 0:
            for i in succ:
                if not nodes[str(i)]['rank_calculated']:
                    # print("couldn't calculate rank")
                    return 1  
            weights = [nodes[str(i)]['rank'] for i in succ]
            # print('node weight:',rank)
            # print('successors:',succ)
            # print('succ weights:',weights)
            max_weights = max(weights)
            # print('max:',max_weights)
            rank += max_weights
        nodes[node]['rank'] = rank
        # print('calculated rank:', rank)
        nodes[node]['rank_calculated'] = True
    return 0

In [21]:
def graph_update():
    count = 0
    for node in reversed(nodes.keys()):
        # print('node:',node)
        count += calculate_rank(node)
    print('not calculated ranks:',count)    
    return count

In [22]:
create_successors()

def p(nodes, i):
    pt = nodes[i]['Data'].split(':')
    return float(pt[0])*3600 + float(pt[1])*60 + float(pt[2])

def change_type_Data(graph):
    for node in graph:
        if type(nodes[node]['Data']) == float:
            break
        pt = nodes[node]['Data'].split(':')
        nodes[node]['Data'] = float(pt[0])*3600 + float(pt[1])*60 + float(pt[2])
    
change_type_Data(nodes)

In [23]:
def rec_calc_rank(nodes, node):
    succ = nodes[node]['Successors']
    if len(succ) == 0 or 'Rank' in nodes[node]:
        nodes[node]['Rank'] = nodes[node]['Data']
        return nodes[node]['Data']
    else:
        m = 0
        for i in succ:
            u = rec_calc_rank(nodes, i)
            if m < u :
                m = u
        nodes[node]['Rank'] = nodes[node]['Data'] + m
        return nodes[node]['Data']
    
rec_calc_rank(nodes, '1')


2843.6552354

Once we have calculated the rank of each node, we can make a priority list which is basically the list of the rank sorted in decreasing order.

In [24]:
def make_priority_list():
    node_list = []
    rank_list = []
    for node in nodes:
        node_list.append(node)
        rank_list.append(nodes[node]['rank'])

    # print(node_list)
    # print(rank_list)
    # indexes = np.flip(np.argsort(rank_list))
    # sorted_nodes = [node_list[i] for i in indexes]
    sorted_nodes = [x for _,x in sorted(zip(rank_list,node_list), reverse=True)]
    return sorted_nodes

In [25]:
def process_graph():
    create_successors()
    add_ranks()
    rec_calc_rank(nodes, '1')
    priority_list = make_priority_list()
    return priority_list
    

In [26]:
start_time = time.time()
priority_list = process_graph()
duration = time.time() - start_time
# print(priority_list)
# print(len(nodes))
# print(len(priority_list))
print('duration:', duration)
print(priority_list)



AttributeError: 'float' object has no attribute 'split'

In [ ]:
def calculate_execution_time(priority_list,n_cores):
    execution_times = [0]*n_cores
    for node in priority_list :
        task_time = weight(nodes[node])
        i = np.argmin(execution_times)
        execution_times[i] += task_time
    return execution_times

In [ ]:
calculate_execution_time(priority_list,2)

[19641.855386000003, 19558.9887496]